In [1]:
using ColoringNames

In [2]:
runnum = "4"
function mklogdir(name)
    logdir = joinpath(pwd(), "logs", name * runnum)
    mkpath(logdir)
    logdir
end

mklogdir (generic function with 1 method)

In [ ]:
function qualitative_demo(mdl, do_oov=true)
    plot_query(mdl,  "brownish green") |> display
    plot_query(mdl,  "brown") |> display
    plot_query(mdl,  "brownish") |> display
    plot_query(mdl,  "greenish") |> display
    plot_query(mdl,  "greenish brown") |> display
    plot_query(mdl,  "green")|> display
    do_oov && plot_query(mdl,  "ish")|> display
end

qualitative_demo(mdl::TermToColorDistributionEmpirical) = qualitative_demo(mdl, false)

In [ ]:
many_names = load_color_nameset()
word_embs, vocab, enc = load_text_embeddings(keep_words=many_names)
cldata = load_munroe_data(dev_as_train=false, dev_as_test=true, encoding_ = enc)

In [ ]:
sowe_unsmoothed = TermToColorDistributionSOWE(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(sowe_unsmoothed, cldata, Val{false}();
        log_dir = mklogdir("sowe_unsmoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )
@show evaluate(sowe_unsmoothed, cldata.dev)

In [ ]:
qualitative_demo(sowe_unsmoothed)

In [ ]:
sowe_smoothed = TermToColorDistributionSOWE(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(sowe_smoothed, cldata, Val{true}();
        log_dir=mklogdir("sowe_smoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )
@show evaluate(sowe_smoothed, cldata.dev)

In [ ]:
rnn_unsmoothed = TermToColorDistributionRNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(rnn_unsmoothed, cldata, Val{false}();
        log_dir = mklogdir("rnn_unsmoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )
@show evaluate(rnn_unsmoothed, cldata.dev)


In [ ]:
qualitative_demo(rnn_unsmoothed)

In [ ]:
rnn_smoothed = TermToColorDistributionRNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(rnn_smoothed, cldata, Val{true}();
        log_dir = mklogdir("rnn_smoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )

@show evaluate(rnn_smoothed, cldata.dev)

In [ ]:
qualitative_demo(rnn_smoothed)

In [ ]:
cnn_unsmoothed = TermToColorDistributionCNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(cnn_unsmoothed, cldata, Val{false}();
        log_dir=mklogdir("cnn_unsmoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )

@show evaluate(cnn_unsmoothed, cldata.dev)

In [ ]:
qualitative_demo(cnn_unsmoothed)

In [ ]:
cnn_smoothed = TermToColorDistributionCNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(cnn_smoothed, cldata, Val{true}();
        log_dir=mklogdir("cnn_smoothed_rand"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )

@show evaluate(cnn_smoothed, cldata.dev)

In [ ]:
qualitative_demo(cnn_smoothed)

In [ ]:
qualitative_demo(cnn_smoothed)

In [ ]:
noml_unsmoothed = TermToColorDistributionEmpirical(256)
train!(noml_unsmoothed, cldata, Val{false}())
@show evaluate(noml_unsmoothed, cldata.dev)

In [ ]:
qualitative_demo(noml_unsmoothed)

In [ ]:
qualitative_demo(noml_smoothed)

In [ ]:
noml_smoothed = TermToColorDistributionEmpirical(256)
train!(noml_smoothed, cldata, Val{true}())
@show evaluate(noml_smoothed, cldata.dev)